# Sieci rekurencyjne (RNN)

Uczymy tagger morfosyntaktyczny, w architekturze many-to-many.
Zadaniem jest więc otagowanie sekwencji tokenów etykietami reprezentującymi części mowy.

Dane pochodzą z konkursu PolEval http://2017.poleval.pl/index.php/tasks/

Pobranie danych:

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/rnn_data.zip?raw=true
!mv rnn_data.zip?raw=true rnn_data.zip
!unzip rnn_data.zip

#### Import pakietów

In [ ]:
import os
import numpy
import torch



#### Wczytywanie danych
Dzielimy dane na train, dev, i test set

In [ ]:
def load_data(path):
    with open(path) as f:
        data_text = f.read()
    docs = data_text.split("\n\n")
    out = []
    for doc in docs:
        tok_list = []
        toks = doc.split("\n")
        for tok in toks:
            form, lemma, tag = tok.split("\t")
            tok_list.append((form, lemma, tag))
        out.append(tok_list)
    return out


def get_label_set(data):
    label_set = set([])
    for doc in data:
        for tok in doc:
            tag = tok[2]
            label_set.add(tag)
    label_list = sorted(list(label_set))
    return label_list

train_data = load_data(os.path.join("rnn_data", "train.tab"))
dev_data = load_data(os.path.join("rnn_data", "dev.tab"))
test_data = load_data(os.path.join("rnn_data", "test.tab"))
label_list = get_label_set(train_data) # lista tagów
label_to_ind = {label_list[i]:i for i in range(len(label_list))} # słownik tag -> numer
num_labels = len(label_list)

print("Kategorie dla klasyfikatora: ", label_list, "\n")
print("Długość zbioru treningowego (liczona w dokumentach): ", len(train_data))
print("Przykładowy dokument:")
for x in train_data[100][:10]:
    print(x)



#### Przygotowanie reprezentacji wektorowych

In [ ]:
!wget http://dsmodels.nlp.ipipan.waw.pl/dsmodels/nkjp+wiki-forms-all-100-cbow-hs.txt.gz
!gunzip nkjp+wiki-forms-all-100-cbow-hs.txt.gz

from gensim.models import KeyedVectors

VEX = KeyedVectors.load_word2vec_format("nkjp+wiki-forms-all-100-cbow-hs.txt")

In [ ]:
kot = VEX["kot"]
num_feats = len(kot)

print(kot)
print(type(kot), kot.dtype, kot.shape)

def w2v(form):
    try:
        return VEX[form.lower()]
    except KeyError:
        return numpy.zeros((VEX.vector_size,))

#### Definicja modelu

In [ ]:
from torch import nn, cat, tanh

class RecurrentModel(nn.Module):
    def __init__(self, input_size, state_size, output_size):# wymiary wejścia, stanu rekurencyjnego, ilośc etykiet
        super(RecurrentModel, self).__init__()
        self.state_size = state_size
        self.recurrent = nn.Linear(input_size + state_size, state_size) # warstwa rekurencyjna
        self.activation = tanh
        self.classifier = nn.Linear(state_size, output_size) # klasyfikator
        self.softmax = nn.LogSoftmax(dim=1)

    def init_state(self): # sztuczny "stan" dla kroku zerowego - macierz zer
        return torch.zeros(1, self.state_size)

    def forward(self, input, state):
        concatenated = cat([input, state], axis=1) # konkatenacja wejścia i stanu
        state = self.activation(self.recurrent(concatenated))
        output = self.softmax(self.classifier(state))
        return output, state # wyjście i stan

#### Definicja funkcji pomocniczych
Definiujemy funkcje do przetwarzania dokumentów na gotowe przykłady treningowe, funkcje trenującą, oraz testującą model na jednym takim przykładzie, oraz funkcje do interpretacji wyjścia z sieci neuronowej, i tagowania zdań w fazie użytkowej.

In [ ]:
from torch import tensor

def doc_to_training_example(doc):
    forms_vec = [] # lista wektorów dla tokenów
    tags_vec = [] # lista numerów dla tagów
    doc_len = len(doc)
    for form, lemma, tag in doc:
        forms_vec.append(w2v(form)) # zbieramy reprezentacje wektorowe tokenów
        tags_vec.append(label_to_ind[tag])
    X = torch.tensor(forms_vec, dtype=torch.float32) # tworzenie tensora z wektorów tokenów
    X = X.reshape(doc_len, 1, num_feats) # zmiana kształtu tak, by dodać trzeci wymiar, na razie to tylko wymóg formalny
    Y = torch.tensor(tags_vec)
    Y = Y.reshape((doc_len, 1)) # tutaj analogiczne przekształcenie, tylko że dodajemy drugi wymiar
    return X, Y


def train_on_example(model, criterion, X, Y):
    model.zero_grad() # zerujemy zakumulowane z poprzednich przykładów gradienty
    state = model.init_state() # inicjalizujemy stan z kroku zero
    loss = 0
    for tok_x, tok_y in zip(X,Y): # iterujemy po tokenach w dokumencie
        output, state = model(tok_x, state) # wpuszczamy dane do modelu, zbieramy wyjście i stan rekurencyjny
        loss += criterion(output, tok_y) # liczymy stratę względem wzorcowej etykiety
    loss.backward() # po przejściu wszystkich tokenów, uruchamiamy wsteczną propagację
    for p in model.parameters():
        p.data.add_(p.grad.data, alpha = -learning_rate) # aktualizujemy parametry sieci, przez dodanie zakumulowanych gradientów
    return loss.item()


def out_to_label(out):
    index = out.topk(1).indices # wybieramy indeks z macierzy, dla której mamy najwyższą wartość
    tag = label_list[index] # zamieniamy indeks na tag
    return tag, index
    
    
def test_on_example(model, X, Y):
    state = model.init_state() # inicjalizujemy stan z kroku zero
    correct_tokens = 0
    total_tokens = 0
    for tok_x, tok_y in zip(X,Y):
        total_tokens += 1
        output, state = model(tok_x, state)
        _, label_index = out_to_label(output)
        if label_index == tok_y:
            correct_tokens += 1
    return correct_tokens, total_tokens    

def tag_sentence(model, sent):
    tokens = sent.split(" ")
    tags = []
    doc_len = len(tokens)
    forms_vec = [w2v(tok) for tok in tokens] # zbieramy reprezentacje wektorowe tokenów
    X = torch.tensor(forms_vec, dtype=torch.float32)# konstruujemy tensor z wektorów dla tokenów
    X = X.reshape(doc_len, 1, num_feats) # przekształcamy tensor zgodnie z wymogami modeli pytorch
    state = model.init_state()
    for tok_x in X:
        output, state = model(tok_x, state)
        label, _ = out_to_label(output)
        tags.append(label)
    return tokens, tags

#### Sposób działania sieci:

W kroku numer i, sieć pobiera na wejściu reprezentację tokenu o indeksie i, oraz stan sieci z kroku i-1. W pierwszym kroku, rolę fikcyjnego "stanu" gra macierz zerowa. Wyjście z pierwszej warstwy jest przepuszczane przez funkcję aktywacji, i zapisywane dla następnego kroku, oraz wpuszczane dalej do klasyfikatora.

In [ ]:
model = RecurrentModel(100, 50, num_labels)
X, Y = doc_to_training_example(train_data[40])
print("Kształt X: ", X.shape)

print("Stan w pierwszym kroku: ")
state = model.init_state()
print(state)
print("Kształt stanu: ", state.shape)
x1 = X[0]
#print("Wektor dla pierwszego tokenu: ", x1)
print("Kształt wektora dla pierwszego tokenu: ", x1.shape)
concatenated = cat([x1, state], axis=1)
print("Kształt wejścia do sieci: ", concatenated.shape)
recurrent_out = model.recurrent(concatenated)
print("Kształt wyjścia z pierwszej warstwy: ", recurrent_out.shape)
state = model.activation(recurrent_out)
output = model.softmax(model.classifier(state))
print("Kształt wyjścia z drugiej warstwy: ", output.shape)

# wykorzystanie wyjścia z pierwszej warstwy w kolejnym kroku
x2 = X[1]
concatenated = cat([x2, state], axis=1)
recurrent_out = model.recurrent(concatenated)
state = model.activation(recurrent_out)
output = model.softmax(model.classifier(state))

#### Trening i ewaluacja modelu

In [ ]:
from tqdm.notebook import tqdm
model = RecurrentModel(100, 50, num_labels)
criterion = torch.nn.NLLLoss() # wybór funkcji straty
# parametry uczenia
learning_rate = 0.005
epochs = 3

for epoch in range(epochs):
    # trening
    total_loss = 0
    for doc in tqdm(train_data):
        X, Y = doc_to_training_example(doc)
        loss = train_on_example(model, criterion, X, Y)
        total_loss += loss
    print(total_loss)
    
    # ewaluacja
    dev_total = 0
    dev_correct = 0
    with torch.no_grad(): # w trakcie ewaluacji należy wyłączyć akumulację gradientów
        for doc in tqdm(dev_data[:400]):
            X, Y = doc_to_training_example(doc)
            correct_tokens, total_tokens = test_on_example(model, X, Y)
            dev_total += total_tokens
            dev_correct += correct_tokens
        accuracy = "{:4.2f}%".format(((dev_correct/dev_total) * 100))
        print("dev acc: ", accuracy)

# test
test_total = 0
test_correct = 0
with torch.no_grad():
    for doc in tqdm(test_data[:400]):
        X, Y = doc_to_training_example(doc)
        correct_tokens, total_tokens = test_on_example(model, X, Y)
        test_total += total_tokens
        test_correct += correct_tokens
    
accuracy = "{:4.2f}%".format(((test_correct/test_total) * 100))
print("test acc: ", accuracy)
        

#### Sprawdzenie modelu na realnych danych

In [ ]:
tokens, tags = tag_sentence(model, "oddychanie to jedna z najważniejszych czynności życiowych")
for token, tag in zip(tokens, tags):
    print(token, tag)